<a href="https://colab.research.google.com/github/imabhinav-singh/Recommender-Systems/blob/main/Implementation_KMeans_and_Genre_Cluster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving googleplaystore_user_reviews.csv to googleplaystore_user_reviews.csv


In [ ]:
import pandas as pd
import numpy as np

def del_duplicates(df):
  df = df.drop_duplicates(keep='last', inplace=False)
  app_data_df['Last Updated'] = pd.to_datetime(app_data_df['Last Updated'])
  df = df.sort_values(by=['App',  'Last Updated'], ascending=False)
  df = df.drop_duplicates(subset='App', keep='first', inplace=False)
  return df

def check_sim(a, b):
  for aword in a:
    for bword in b:
      if aword.strip() == bword.strip():
        return True
  return False

app_data_df = pd.read_csv('googleplaystore.csv')

for row in app_data_df[app_data_df.isnull().any(axis=1)].index:
  app_data_df = app_data_df.drop(row)
app_data_df = app_data_df.reset_index(drop=True)

app_data_df.drop(app_data_df[app_data_df['App'] == 'Life Made WI-Fi Touchscreen Photo Frame'].index, inplace=True)
app_data_df = del_duplicates(app_data_df)
app_data_df = app_data_df.reset_index(drop=True)

distinct_data = sorted(list(set(app_data_df['Genres'].unique())))

dissim_mat = np.empty([len(distinct_data), len(distinct_data)])
dissim_mat.fill(1)

for i in range(len(distinct_data)):
  for j in range(len(distinct_data)):
    rg = distinct_data[i].split(';')
    cg = distinct_data[j].split(';')
    if check_sim(rg, cg):
      dissim_mat[i][j] = 0

In [ ]:
cluster_centres_list = ['Art & Design', 'Auto & Vehicles', 'Beauty', 'Books & Reference', 'Business', 'Comics', 'Communication', 
                        'Dating', 'Education', 'Entertainment', 'Events', 'Finance', 'Food & Drink', 'Health & Fitness', 'House & Home', 
                        'Libraries & Demo', 'Lifestyle', 'Maps & Navigation', 'Medical', 'News & Magazines', 'Parenting', 'Personalization', 
                        'Photography', 'Productivity', 'Shopping', 'Social', 'Sports', 'Travel & Local', 'Tools', 'Weather', 'Video Players & Editors']

In [ ]:
from sklearn import datasets
import matplotlib.pyplot as plt
import numpy as np
import random

def calc_dist(x, y):
  return dissim_mat[distinct_data.index(x)][distinct_data.index(y)]

def check_eq(x, y):
  for i in range(len(x)):
    if calc_dist(x[i], y[i]) != 0.0:
      return False
  return True

def closest_center_ind(x, centres):
  min_ind = 0  
  for i in range(1, len(centres)):
    if calc_dist(x, centres[i]) < calc_dist(x, centres[min_ind]):
      min_ind = i  
  return min_ind

def find_centroid(arr):
  cat_dissim = {}
  #print(len(arr))
  for elt in arr:
    if cat_dissim.get(elt) == None:
      cat_dissim[elt] = 0
      for eltj in arr:
        if dissim_mat[distinct_data.index(elt)][distinct_data.index(eltj)] == 1:
          cat_dissim[elt] += -1
  return max(cat_dissim, key=cat_dissim.get)

def fit_kmeans(k, X, max_itr=50):
  rand_ind = np.random.choice(range(len(cluster_centres_list)), k, replace=False)
  cluster_centres = [cluster_centres_list[rand_ind[i]] for i in range(k)]
  cluster_label = [0 for i in range(len(X))]

  for itr in range(max_itr):
    prev_cent = cluster_centres.copy()
    for i in range(len(X)):
      cluster_label[i] = closest_center_ind(X[i], cluster_centres)

    for i in range(k):
      cluster = []
      for j in range(len(X)):
        if cluster_label[j] == i:
          cluster.append(X[j])
      cluster_centres[i] = find_centroid(cluster)
      #print(find_centroid(cluster))

    if check_eq(cluster_centres, prev_cent):
      #print('Converged @ '+str(itr))
      break

  return cluster_centres, cluster_label

In [ ]:
X = np.asarray(app_data_df['Genres'])

In [ ]:
cluster_centres, cluster_label = fit_kmeans(31, X, 50)
clusters = {}
for label in range(31):
  clusters[label] = []
for i in range(len(X)):
  clusters[cluster_label[i]].append(X[i])

for col in clusters:
  print('Label '+str(col)+' Length '+str(len(clusters[col])))
  clusters[col] = pd.DataFrame(clusters[col])
  #clusters[col].to_excel(str(col)+'_cluster.xlsx')
  display(clusters[col])

Label 0 Length 1752


,0
0,Simulation
1,Adventure
2,Strategy
3,Action
4,Arcade
...,...
1747,Casual
1748,Puzzle
1749,Action
1750,Board


Label 1 Length 296


,0
0,Personalization
1,Personalization
2,Personalization
3,Personalization
4,Personalization
...,...
291,Personalization
292,Personalization
293,Personalization
294,Personalization


Label 2 Length 547


...

,0
0,Books & Reference
1,Books & Reference
2,Books & Reference
3,Books & Reference
4,Books & Reference
...,...
164,Books & Reference
165,Books & Reference
166,Books & Reference
167,Books & Reference


Label 29 Length 180


,0
0,Shopping
1,Shopping
2,Shopping
3,Shopping
4,Shopping
...,...
175,Shopping
176,Shopping
177,Shopping
178,Shopping


Label 30 Length 61


,0
0,House & Home
1,House & Home
2,House & Home
3,House & Home
4,House & Home
...,...
56,House & Home
57,House & Home
58,House & Home
59,House & Home


In [ ]:
app_data_df['Installs'] = app_data_df.Installs.str.replace('+', '')
app_data_df['Installs'] = app_data_df.Installs.str.replace(',', '')
app_data_df['Price'] = app_data_df.Price.str.replace('$','')

from sklearn.preprocessing import MinMaxScaler 
scaler = MinMaxScaler()

scaler.fit(app_data_df[['Rating']])
app_data_df['Rating'] = scaler.transform(app_data_df[['Rating']])

scaler.fit(app_data_df[['Price']])
app_data_df['Price'] = scaler.transform(app_data_df[['Price']]) 

scaler.fit(app_data_df[['Installs']])
app_data_df['Installs'] = scaler.transform(app_data_df[['Installs']]) 

scaler.fit(app_data_df[['Reviews']])
app_data_df['Reviews'] = scaler.transform(app_data_df[['Reviews']])

app_org_data_df = app_data_df[['App', 'Rating', 'Reviews', 'Installs', 'Price', 'Genres']]
app_data_df = app_data_df[['Rating', 'Reviews', 'Installs', 'Price']]

display(app_data_df)

,Rating,Reviews,Installs,Price
0,0.925,1.491844e-04,9.999990e-04,0.000000
1,0.750,2.303018e-07,9.999000e-06,0.000000
2,0.900,2.827211e-04,9.999990e-04,0.000000
3,0.700,9.173689e-06,9.999000e-06,0.999975
4,0.850,2.418169e-06,9.999000e-06,0.000000
...,...,...,...,...
8185,0.850,3.326582e-07,9.900000e-08,0.000000
8186,0.925,7.318480e-06,9.999000e-06,0.000000
8187,0.875,3.300993e-06,9.999000e-06,0.000000
8188,0.625,1.458578e-06,9.999000e-06,0.000000


In [ ]:
from sklearn.cluster import KMeans

X = app_data_df.values
kmeans = KMeans(n_clusters=7, init='k-means++', max_iter=300, n_init=10, random_state=0)
pred_y = kmeans.fit_predict(X)
print(kmeans.labels_)

[5 6 5 ... 1 0 1]


In [ ]:
print(kmeans.cluster_centers_)
app_df_labels = app_data_df.copy()
app_df_labels.insert(4, 'Label', kmeans.labels_)
display(app_df_labels)

app_clusters = {}
for i in range(7):
  app_clusters[i] = app_org_data_df.iloc[app_df_labels[app_df_labels['Label'] == i].index]

[[ 5.84725686e-01  1.03202943e-04  1.05658558e-03  9.46539900e-04]
 [ 8.37921184e-01  3.53797794e-03  8.55326028e-03  7.22092001e-04]
 [ 8.25568182e-01  1.85136150e-01  7.27272727e-01 -1.30104261e-18]
 [ 3.17200855e-01  1.10407634e-05  8.21655599e-05  6.40277778e-04]
 [ 7.16666667e-01  7.70572835e-06  1.46056667e-05  9.78310000e-01]
 [ 9.32734237e-01  3.09243100e-03  4.28473219e-03  1.26913671e-03]
 [ 7.37695214e-01  6.16922451e-04  2.92815909e-03  5.16750630e-04]]


,Rating,Reviews,Installs,Price,Label
0,0.925,1.491844e-04,9.999990e-04,0.000000,5
1,0.750,2.303018e-07,9.999000e-06,0.000000,6
2,0.900,2.827211e-04,9.999990e-04,0.000000,5
3,0.700,9.173689e-06,9.999000e-06,0.999975,4
4,0.850,2.418169e-06,9.999000e-06,0.000000,1
...,...,...,...,...,...
8185,0.850,3.326582e-07,9.900000e-08,0.000000,1
8186,0.925,7.318480e-06,9.999000e-06,0.000000,5
8187,0.875,3.300993e-06,9.999000e-06,0.000000,1
8188,0.625,1.458578e-06,9.999000e-06,0.000000,0


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix

def cleanhtml(sentence): #function to clean the word of any html-tags
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext
def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return cleaned


app_review_df = pd.read_csv('/content/googleplaystore_user_reviews.csv')
app_review_df = app_review_df.dropna()
reviews = app_review_df['Translated_Review']
labels = app_review_df['Sentiment']

Stop = set(stopwords.words('english'))
WrdLem = WordNetLemmatizer()
#Cleaning the reviews(removing html tags,punctuation,Lemmatizations) 
nltk.download('wordnet')
Cleaned_sent=[]
for sent in reviews:
    r1=[]
    sent=cleanhtml(sent)
    sent=cleanpunc(sent)
    sent=sent.lower()
    for  word in sent.split():
        if ((word.isalpha()) & (len(word)>2)):
            if word not in Stop:
                w=WrdLem.lemmatize(word)
                r1.append(w)
            else:
                continue
        else:
            continue
    str1 = (" ".join(r1))        
     
    Cleaned_sent.append(str1)

app_review_df['Cleaned_text'] = Cleaned_sent
app_review_df = app_review_df.reset_index(drop=True)
display(app_review_df)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity,Cleaned_text
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.000000,0.533333,like eat delicious food thats cooking food cas...
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.250000,0.288462,help eating healthy exercise regular basis
2,10 Best Foods for You,Works great especially going grocery store,Positive,0.400000,0.875000,work great especially going grocery store
3,10 Best Foods for You,Best idea us,Positive,1.000000,0.300000,best idea
4,10 Best Foods for You,Best way,Positive,1.000000,0.300000,best way
...,...,...,...,...,...,...
37422,Housing-Real Estate & Property,Most ads older many agents ..not much owner po...,Positive,0.173333,0.486667,ad older many agent much owner posted detail r...
37423,Housing-Real Estate & Property,"If photos posted portal load, fit purpose. I'm...",Positive,0.225000,0.447222,photo posted portal load fit purpose sure stor...
37424,Housing-Real Estate & Property,"Dumb app, I wanted post property rent give opt...",Negative,-0.287500,0.250000,dumb app wanted post property rent give option...
37425,Housing-Real Estate & Property,I property business got link SMS happy perform...,Positive,0.800000,1.000000,property business got link sm happy performanc...


In [ ]:
app_review_copy = app_review_df.copy()
for app in app_review_df['App']:
  if app not in list(app_org_data_df['App']):
    app_review_copy.drop(app_review_copy[app_review_copy['App'] == app].index, inplace=True)
    app_review_copy = app_review_copy.reset_index(drop=True)
app_review_df = app_review_copy
display(app_review_copy)

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity,Cleaned_text
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.000000,0.533333,like eat delicious food thats cooking food cas...
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.250000,0.288462,help eating healthy exercise regular basis
2,10 Best Foods for You,Works great especially going grocery store,Positive,0.400000,0.875000,work great especially going grocery store
3,10 Best Foods for You,Best idea us,Positive,1.000000,0.300000,best idea
4,10 Best Foods for You,Best way,Positive,1.000000,0.300000,best way
...,...,...,...,...,...,...
35924,Housing-Real Estate & Property,Most ads older many agents ..not much owner po...,Positive,0.173333,0.486667,ad older many agent much owner posted detail r...
35925,Housing-Real Estate & Property,"If photos posted portal load, fit purpose. I'm...",Positive,0.225000,0.447222,photo posted portal load fit purpose sure stor...
35926,Housing-Real Estate & Property,"Dumb app, I wanted post property rent give opt...",Negative,-0.287500,0.250000,dumb app wanted post property rent give option...
35927,Housing-Real Estate & Property,I property business got link SMS happy perform...,Positive,0.800000,1.000000,property business got link sm happy performanc...


In [ ]:
def filter_ratings(df, app):
  rate_app = df.loc[df['App'] == app].values[0][1]
  return df.loc[df['Rating'] >= rate_app]

def filter_genre(df, genre, app):
  genre = set(genre[0])
  frames = []
  for g in genre:
    frames.append(df.loc[df['Genres'] == g])
  return pd.concat(frames)

a, b = 0, 0
X = app_review_df.values
recommended_apps = [[] for i in range(len(X))]
#recommended_apps_df = [[] for i in range(len(X))]
for user_id in range(len(X)):
  print(user_id)
  label_kmeans = kmeans.labels_[app_org_data_df[app_org_data_df['App'] == X[user_id][0]].index[0]]
  label_genre = cluster_label[app_org_data_df[app_org_data_df['App'] == X[user_id][0]].index[0]]
  if X[user_id][2] == 'Negative':
    rec_set = filter_ratings(filter_genre(app_clusters[label_kmeans], clusters[label_genre], X[user_id][0]), X[user_id][0])
    recommended_apps[user_id].append(rec_set.values[:,0])
    #recommended_apps_df[user_id] = rec_set.reset_index(drop=True)
    a += 1
  else:
    rec_set = filter_genre(app_clusters[label_kmeans], clusters[label_genre], X[user_id][0])
    recommended_apps[user_id].append(rec_set.values[:,0])
    #recommended_apps_df[user_id] = rec_set.reset_index(drop=True)
    b += 1
print(a, b)

In [ ]:
cov_set = set()
for i in range(len(X)):
  for j in range(len(recommended_apps[i][0])):
    cov_set.add(recommended_apps[i][0][j])
print(len(cov_set))

7637


In [ ]:
def cal_sim(x, y):
  dissim = (x[1] - y[1]) + (x[2] - y[2]) + (x[3] - y[3]) + (x[0] - y[0])
  dissim /= 4
  return 1 - dissim

sim_mat = np.zeros((len(app_data_df), len(app_data_df)))
for i in range(len(app_data_df)):
  for j in range(len(app_data_df)):
    if i > j:
      break
    sim_mat[i][j] = cal_sim(app_data_df.loc[i], app_data_df.loc[j])
    sim_mat[j][i] = sim_mat[i][j]
sim_mat_df = pd.DataFrame(sim_mat, index=app_org_data_df['App'], columns=app_org_data_df['App'])

In [ ]:
ils = np.zeros(len(X))
for i in range(len(X)):
  if len(recommended_apps[i][0]) <= 1:
    continue
  ils_mat = np.zeros((len(recommended_apps[i][0]), len(recommended_apps[i][0])))
  for j in range(len(recommended_apps[i][0])):
    for k in range(len(recommended_apps[i][0])):
      ils_mat[j][k] = sim_mat_df[recommended_apps[i][0][j]][recommended_apps[i][0][k]]
  ils[i] = np.sum(ils_mat) / (len(recommended_apps[i][0])*len(recommended_apps[i][0]) - len(recommended_apps[i][0]))
avg_ils = np.sum(ils) / len(X)
print(avg_ils)

0.1624137542
